In [24]:
%load_ext autoreload
%autoreload 2

from common import add_workspace_to_path

add_workspace_to_path()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
from shared.secrets import get_secret, SecretKeys
from infrastructure.google.GoogleSecret import GoogleSecret

In [26]:
credentials_model = get_secret(SecretKeys.GoogleCredentials, GoogleSecret)

In [27]:
from google.oauth2.credentials import Credentials

credentials = Credentials.from_authorized_user_info(credentials_model.credentials.model_dump())

In [28]:
from datetime import datetime, timezone
from googleapiclient.discovery import build

class TaskService:

    def __init__(self, credentials):
        self.credentials = credentials
        self.service = build(
            "tasks", "v1", credentials=credentials, cache_discovery=False
        )

    def get_default_tasklist(self):
        """Gets the default task list ID."""
        tasklists = self.service.tasklists().list().execute()
        return tasklists["items"][0]["id"]  # Get the first (default) task list

    def create_task_with_notes(self, tasklist_id, title, notes, due_date=None):
        """Creates a new task in the specified task list with notes."""
        if due_date is None:
            # Set due date to end of today
            due_date = datetime.now(timezone.utc).replace(
                hour=23, minute=59, second=59, microsecond=0
            )

        task_body = {
            "title": title,
            "notes": notes,
            "due": due_date.isoformat(),
        }
        task = (
            self.service.tasks().insert(tasklist=tasklist_id, body=task_body).execute()
        )
        return task

In [29]:
service = TaskService(credentials)
default_tasklist_id = service.get_default_tasklist()

In [31]:
service.create_task_with_notes(
    default_tasklist_id,
    "Sample Task Title",
    "This is a sample task created via the Google Tasks API.",
)

{'kind': 'tasks#task',
 'id': 'dlpidUpwcWpWZEVSWGlZNg',
 'etag': '"lSgN3jJP4bE"',
 'title': 'Sample Task Title',
 'updated': '2025-10-18T20:17:10.454Z',
 'selfLink': 'https://www.googleapis.com/tasks/v1/lists/MDY0Mzc2NjgyMDc4MTc0Nzg3Mjk6MDow/tasks/dlpidUpwcWpWZEVSWGlZNg',
 'position': '00000000000000000000',
 'notes': 'This is a sample task created via the Google Tasks API.',
 'status': 'needsAction',
 'due': '2025-10-18T00:00:00.000Z',
 'links': [],
 'webViewLink': 'https://tasks.google.com/task/vZbuJpqjVdERXiY6?sa=6'}